### This is a notebook to run the preprocessing pipeline prior to uploading your TIFs to deepcell

In [ ]:
# import required packages
import os
import numpy as np
import skimage.io as io
import xarray as xr

from segmentation.utils import data_utils

## This script is currently configured as a template to run with the provided example data. If running your own data, make a copy of this notebook first before modifying it.                         Go to file-> make a copy to create a copy of this notebook

### <span style="color:#ff0000"> All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data' regardless of if it's input or output. </span>

In [ ]:
# load TIFs from GUI-based directory structure
data_dir = "../data/example_dataset/input_data/"

# data format
MIBItiff = False
MIBItiff_suffix = "-MassCorrected-Filtered.tiff"

# nuclear channel name(s) (or None)
nucs = ['HH3']

# membrane channel name(s) (or None)
mems = ['Membrane']

# validate data_dir
data_utils.validate_paths(data_dir)

# either get all points in the folder
points = os.listdir(data_dir)
points = [point for point in points if os.path.isdir(data_dir + point) and point.startswith("Point")]
if MIBItiff:
    points = [point for point in points if point.endswith(MIBItiff_suffix)]

# optionally, select a specific set of points manually
# points = ["Point1", "Point2"]

# TODO: MIBItiff manual selection

In [ ]:
# load channels to be included in deepcell data
channels = (nucs if nucs else []) + (mems if mems else [])
if MIBItiff:
    data_xr = data_utils.load_imgs_from_mibitiff([os.path.join(data_dir, pnt) for pnt in points], channels=channels)
else:
    data_xr = data_utils.load_imgs_from_tree(data_dir, img_sub_folder="TIFs", fovs=points, imgs=channels)

# TODO: Add metadata for channel name (no need for fixed-order channels)
for fov in data_xr.fovs.values:
    out = np.zeros((data_xr.shape[1], data_xr.shape[2], 2), dtype="uint8")

    # sum over nuclear channels and add to output
    for nuc in nucs:
        out[:, :, 0] += data_xr.loc[fov, :, :, nuc].values.astype("uint8")
    for mem in mems:
        out[:, :, 1] += data_xr.loc[fov, :, :, mem].values.astype("uint8")

    save_path = os.path.join(data_dir, f'{fov}_deepcell_input.tif')
    io.imsave(save_path, out, plugin='tifffile')

# save xarray for running through deepcell
data_xr.to_netcdf(os.path.join(data_dir, "deepcell_input.xr"), format="NETCDF3_64BIT")